In [1]:
import os

import numpy as np
import PIL.Image

import tensorflow as tf
from tensorflow.python.compiler.tensorrt import trt_convert as trt

In [2]:
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    for device in physical_devices:
        tf.config.experimental.set_memory_growth(device, True)
        print('{} memory growth: {}'.format(device, tf.config.experimental.get_memory_growth(device)))
else:
    print("Not enough GPU hardware devices available")

PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU') memory growth: True


In [3]:
MODEL_BASE_PATH = '/workspace/models/'

In [4]:
class MobileNetV2Model(tf.keras.Model):
    def __init__(self, name: str):
        shape = (224, 224, 3)
        base_model = tf.keras.applications.MobileNetV2(input_shape=shape, include_top=True, weights='imagenet')
        inputs = tf.keras.Input(shape)
        outputs = base_model(inputs)
        
        super().__init__(inputs=inputs, outputs=outputs, name=name)

    @tf.function(
        input_signature=[tf.TensorSpec(shape=[None, 224, 224, 3], dtype=tf.float32, name="imgs")]
    )
    def serving_fn(self, imgs: tf.Tensor) -> tf.Tensor:
        return self(imgs)

    def save(self):
        tf_saved_model_path = os.path.join(MODEL_BASE_PATH, self.name, '0')
        
        signatures = {"serving_default": self.serving_fn}
        tf.saved_model.save(self, tf_saved_model_path, signatures=signatures)
        
        params = trt.DEFAULT_TRT_CONVERSION_PARAMS._replace(
            precision_mode=trt.TrtPrecisionMode.FP16,
            max_batch_size=10
        )
        converter = trt.TrtGraphConverterV2(input_saved_model_dir=tf_saved_model_path, conversion_params=params)
        converter.convert()
        converter.save(os.path.join(MODEL_BASE_PATH, '{}_trt'.format(model.name), '0'))

In [5]:
model = MobileNetV2Model('test1')

14540800/14536120 [==============================] - 1s 0us/step


In [6]:
model.save()

INFO:tensorflow:Assets written to: ../models/test1/0/assets
INFO:tensorflow:Linked TensorRT version: (7, 1, 3)
INFO:tensorflow:Loaded TensorRT version: (7, 1, 3)
INFO:tensorflow:Could not find TRTEngineOp_0_0 in TF-TRT cache. This can happen if build() is not called, which means TensorRT engines will be built and cached at runtime.
INFO:tensorflow:Assets written to: ../models/test1_trt/0/assets


In [2]:
!ls /workspace/models/*.h5

/workspace/models/yolov3.h5


In [54]:
class YOLOV3Model(tf.keras.Model):
    def __init__(self, name: str, path: str):
        self.input_height = 608
        self.input_width = 608
        
        self.num_classes = 80
        
        self.anchors = tf.constant([
            [
                [10, 13],
                [16, 30],
                [33, 23]
            ], [
                [30, 61],
                [62, 45],
                [59, 119]
            ], [
                [116, 90],
                [156, 198],
                [373, 326],
            ]
        ], dtype=tf.float32)
        
        shape = (self.input_height, self.input_width, 3)
        base_model = tf.keras.models.load_model(path)
        inputs = tf.keras.Input(shape)
        outputs = base_model(inputs)
        
        super().__init__(inputs=inputs, outputs=outputs, name=name)
    
    @tf.function
    def decode(self, output: tf.Tensor, anchor: tf.Tensor):
        batch_size = tf.shape(output)[0]
        output_height = tf.shape(output)[1]
        output_width = tf.shape(output)[2]

        output = tf.reshape(output, (batch_size, output_height, output_width, 3, 5 + self.num_classes))

        xy_grid = tf.meshgrid(tf.range(output_height), tf.range(output_width))
        xy_grid = tf.stack(xy_grid, -1)[:, :, tf.newaxis]
        xy_grid = tf.tile(xy_grid[tf.newaxis, ...], [batch_size, 1, 1, 3, 1])

        xy_grid = tf.cast(xy_grid, tf.float32)

        dxdy_activated = tf.sigmoid(output[:, :, :, :, :2])
        dwdh = output[:, :, :, :, 2:4]
        
        xy = ((dxdy_activated - 0.5) + xy_grid + 0.5) / tf.cast([output_width, output_height], tf.float32)
        wh_half = tf.exp(dwdh) * anchor / (2 * tf.cast([self.input_width, self.input_height], tf.float32))

        output = tf.concat(
            [
                xy - wh_half,
                xy + wh_half,
                tf.sigmoid(output[:, :, :, :, 4:]),
            ], -1
        )

        return tf.reshape(output, (batch_size, -1, 4 + 1 + self.num_classes))

    @tf.function(
        input_signature=[tf.TensorSpec(shape=[None, 608, 608, 3], dtype=tf.float32, name="imgs")]
    )
    def serving_fn(self, imgs: tf.Tensor) -> tf.Tensor:
        l, s, m = self(imgs)
        
        s = self.decode(s, self.anchors[0])
        m = self.decode(m, self.anchors[1])
        l = self.decode(l, self.anchors[2])
        
        res = tf.concat((s, m, l), 1)
        
        boxes, confidences, class_probabilities = tf.split(res, [4, 1, self.num_classes], -1)
        
        scores = confidences * class_probabilities
        
        return tf.image.combined_non_max_suppression(
            tf.reshape(boxes, (tf.shape(boxes)[0], -1, 1, 4)), scores,
            50, 50,
            0.5, 0.4,
            clip_boxes=False
        )

    def save(self):
        tf_saved_model_path = os.path.join(MODEL_BASE_PATH, self.name, '0')
        
        signatures = {"serving_default": self.serving_fn}
        tf.saved_model.save(self, tf_saved_model_path, signatures=signatures)
        
        params = trt.DEFAULT_TRT_CONVERSION_PARAMS._replace(
            precision_mode=trt.TrtPrecisionMode.FP16,
            max_batch_size=10
        )
        converter = trt.TrtGraphConverterV2(input_saved_model_dir=tf_saved_model_path, conversion_params=params)
        converter.convert()
        converter.save(os.path.join(MODEL_BASE_PATH, '{}_trt'.format(model.name), '0'))

In [55]:
model = YOLOV3Model('yolov3', os.path.join(MODEL_BASE_PATH, 'yolov3.h5'))

In [56]:
x = tf.random.uniform((2, 608, 608, 3))

In [57]:
model.serving_fn(x)

CombinedNonMaxSuppression(nmsed_boxes=<tf.Tensor: shape=(2, 50, 4), dtype=float32, numpy=
array([[[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],


In [76]:
model.save()

INFO:tensorflow:Assets written to: /workspace/models/yolov3/0/assets
INFO:tensorflow:Linked TensorRT version: (7, 1, 3)
INFO:tensorflow:Loaded TensorRT version: (7, 1, 3)
INFO:tensorflow:Could not find TRTEngineOp_1_2 in TF-TRT cache. This can happen if build() is not called, which means TensorRT engines will be built and cached at runtime.
INFO:tensorflow:Could not find TRTEngineOp_1_0 in TF-TRT cache. This can happen if build() is not called, which means TensorRT engines will be built and cached at runtime.
INFO:tensorflow:Could not find TRTEngineOp_1_13 in TF-TRT cache. This can happen if build() is not called, which means TensorRT engines will be built and cached at runtime.
INFO:tensorflow:Could not find TRTEngineOp_1_1 in TF-TRT cache. This can happen if build() is not called, which means TensorRT engines will be built and cached at runtime.
INFO:tensorflow:Could not find TRTEngineOp_1_5 in TF-TRT cache. This can happen if build() is not called, which means TensorRT engines will 

In [74]:
img = tf.image.decode_image(tf.io.read_file('sf.jpg'), 3)
img_letter_box = tf.image.resize_with_pad(img, 608, 608)
img_normalized = img_letter_box / 255.0
imgs = img_normalized[tf.newaxis, ...]

In [75]:
model.serving_fn(imgs)

CombinedNonMaxSuppression(nmsed_boxes=<tf.Tensor: shape=(1, 50, 4), dtype=float32, numpy=
array([[[ 0.74796623,  0.75367075,  0.99423736,  0.8824597 ],
        [ 0.6754285 ,  0.7606683 ,  0.7079605 ,  0.8015331 ],
        [ 0.59259444,  0.78136146,  0.6032229 ,  0.80154324],
        [ 0.5949346 ,  0.7775313 ,  0.6050073 ,  0.7981953 ],
        [ 0.5222986 ,  0.7841931 ,  0.53417575,  0.8026394 ],
        [ 0.630662  ,  0.73028594,  0.7259639 ,  0.84273535],
        [-0.02498848,  0.7493981 ,  0.0684564 ,  0.87260294],
        [ 0.04682338,  0.75067437,  0.15263286,  0.87288594],
        [ 0.4735718 ,  0.72715735,  0.59456336,  0.7666298 ],
        [ 0.44052532,  0.61213034,  0.5255808 ,  0.6901886 ],
        [ 0.51553476,  0.7402755 ,  0.62544334,  0.8489164 ],
        [ 0.5482995 ,  0.7234647 ,  0.65252763,  0.85085964],
        [ 0.49022123,  0.6886039 ,  0.57789904,  0.8040917 ],
        [ 0.5175544 ,  0.7837814 ,  0.53068054,  0.8029568 ],
        [ 0.09577223,  0.80272686,  0.1065